In [ ]:
# ======================================================================================================
# PROBLEM A3
#
# Build a classifier for the Human or Horse Dataset with Transfer Learning.
# The test will expect it to classify binary classes.
# Note that all the layers in the pre-trained model are non-trainable.
# Do not use lambda layers in your model.
#
# The horse-or-human dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).
# Inception_v3, pre-trained model used in this problem is developed by Google.
#
# Desired accuracy and validation_accuracy > 97%.
# =======================================================================================================
# Luv speedrunning.

In [1]:
#Check for GPU
import tensorflow as tf
print(tf.__version__)
print(tf.test.is_gpu_available())

#python version
import sys
sys.version

2.13.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


2024-02-15 08:30:44.010402: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-15 08:30:44.232577: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-15 08:30:44.232646: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-15 08:30:44.728829: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-15 08:30:44.728990: I tensorflow/compile

'3.9.18 (main, Sep 11 2023, 13:41:44) \n[GCC 11.2.0]'

In [2]:
#Libraries
import urllib.request
import zipfile
from keras_preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras.optimizers import RMSprop
from keras import layers
from keras import Model
from keras.applications.inception_v3 import InceptionV3
import os

In [ ]:
#Get images train and validation

data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
local_file = 'horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('data/horse-or-human')
zip_ref.close()

data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
local_file = 'validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('data/validation-horse-or-human')
zip_ref.close()

train_dir = 'data/horse-or-human'
validation_dir = 'data/validation-horse-or-human'

In [3]:
#const
BATCH_SIZE=32
WIDTH=150
HEIGHT=150
AUTOTUNE=tf.data.AUTOTUNE

In [4]:
#Donwload the weight of the model
inceptionv3 = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
urllib.request.urlretrieve(
    inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [5]:
#get model InceptionV3 and load weights and set trainable false
pre_trained_model = InceptionV3(input_shape=(HEIGHT,WIDTH,3), weights=None, include_top=False)      #Not wanting the final dense layer

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

2024-02-15 08:31:11.903597: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-15 08:31:11.903895: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-15 08:31:11.903962: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-15 08:31:11.904912: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-15 08:31:11.905052: I tensorflow/compile

In [6]:
## Look at pre_trained_model composition
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 74, 74, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 74, 74, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 74, 74, 32)           0         ['batch_normalizati

In [7]:
# Get last layer
last_layer=pre_trained_model.get_layer("mixed10")

In [8]:
#Specify path for data training and validation
TRAINING_DIR = 'data/horse-or-human'
VALIDATION_DIR= 'data/validation-horse-or-human'

In [9]:
#Image Augmentation
train_datagen=ImageDataGenerator(rescale=1./255,
                                rotation_range=5,  # rotation
                                width_shift_range=0.2,  # horizontal shift
                                zoom_range=0.2,  # zoom
                                horizontal_flip=True,  # horizontal flip
                                brightness_range=[0.2,0.8])  # brightness

val_datagen=ImageDataGenerator(rescale=1./255)

In [10]:
#Get data
train_gen= train_datagen.flow_from_directory(
    directory=TRAINING_DIR,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_gen= val_datagen.flow_from_directory(
    directory=VALIDATION_DIR,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.


Found 256 images belonging to 2 classes.


In [11]:
#Get classes
target=train_gen.class_indices.keys()
target

dict_keys(['horses', 'humans'])

In [12]:
# build a model, i think?
def get_model():
    ## continuing pre trained model last layer and flatten it!
    x = layers.Flatten()(last_layer.output)
    

    x = layers.Dense(128, activation="relu")(x)    
    x = layers.Dropout(0.2)(x)
    ## Binary classification
    x = layers.Dense(1, activation='sigmoid')(x)

    model = Model(pre_trained_model.input, x)

    model.compile(optimizer=RMSprop(learning_rate=0.0001),
                    loss='binary_crossentropy',
                    metrics=['acc'])
    return model

In [13]:
model=get_model()
model.fit(train_gen, validation_data=val_gen, epochs=10)

Epoch 1/10


2024-02-15 00:22:36.385218: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-02-15 00:22:37.074566: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-15 00:22:38.361256: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-02-15 00:22:40.011428: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe0aeaaa950 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-15 00:22:40.011507: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-02-15 00:22:40.079035: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-15 00:

33/33 [==============================] - 25s 480ms/step - loss: 0.2176 - acc: 0.9445 - val_loss: 0.2089 - val_acc: 0.9141
Epoch 2/10
33/33 [==============================] - 15s 446ms/step - loss: 0.0793 - acc: 0.9776 - val_loss: 0.0029 - val_acc: 1.0000
Epoch 3/10
33/33 [==============================] - 13s 395ms/step - loss: 0.0253 - acc: 0.9932 - val_loss: 0.0046 - val_acc: 0.9961
Epoch 4/10
33/33 [==============================] - 11s 332ms/step - loss: 0.0441 - acc: 0.9854 - val_loss: 0.0099 - val_acc: 0.9961
Epoch 5/10
33/33 [==============================] - 11s 327ms/step - loss: 0.0200 - acc: 0.9942 - val_loss: 0.0272 - val_acc: 0.9961
Epoch 6/10
33/33 [==============================] - 11s 330ms/step - loss: 0.0269 - acc: 0.9922 - val_loss: 0.0089 - val_acc: 0.9961
Epoch 7/10
33/33 [==============================] - 11s 341ms/step - loss: 0.0171 - acc: 0.9932 - val_loss: 0.0015 - val_acc: 1.0000
Epoch 8/10
33/33 [==============================] - 9s 282ms/step - loss: 0.0267

In [10]:
# ======================================================================================================
# PROBLEM A3
#
# Build a classifier for the Human or Horse Dataset with Transfer Learning.
# The test will expect it to classify binary classes.
# Note that all the layers in the pre-trained model are non-trainable.
# Do not use lambda layers in your model.
#
# The horse-or-human dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).
# Inception_v3, pre-trained model used in this problem is developed by Google.
#
# Desired accuracy and validation_accuracy > 97%.
# =======================================================================================================

import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras.optimizers import RMSprop
from keras import layers
from keras import Model
from keras.applications.inception_v3 import InceptionV3


def solution_A3():
    
    #Download weights
    inceptionv3 = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    urllib.request.urlretrieve(
        inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
    local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    
    #Const
    IMG_HEIGHT = 150
    IMG_WIDTH = 150
    BATCH_SIZE = 32
    AUTOTUNE = tf.data.AUTOTUNE
    
    #Get pre-trained model
    '''Pre trained model doesn't use the default output, and donwloaded weights are set on the model'''
    pre_trained_model =  InceptionV3(include_top=False, weights=local_weights_file, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    for layer in pre_trained_model.layers:
        layer.trainable = False                                     #Set all layers untrainable
    last_layer =  pre_trained_model.get_layer("mixed10")           #Get the last layer of the pre_trained_model
    
    #Get datasets
    data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
    urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
    local_file = 'horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/horse-or-human')
    zip_ref.close()
    data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
    urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
    local_file = 'validation-horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/validation-horse-or-human')
    zip_ref.close()
    
    #Path of dataset dirs
    train_dir = 'data/horse-or-human'
    val_dir = 'data/validation-horse-or-human'

    #Image Augmentation
    train_datagen=ImageDataGenerator(rescale=1./255,
                                    rotation_range=5,           # rotation
                                    width_shift_range=0.2,      # horizontal shift
                                    zoom_range=0.2,             # zoom
                                    horizontal_flip=True,       # horizontal flip
                                    brightness_range=[0.2,0.8]) # brightness
    val_datagen=ImageDataGenerator(rescale=1./255)
    
    #Get datasets
    train_gen=train_datagen.flow_from_directory(directory=train_dir, target_size=(IMG_HEIGHT, IMG_WIDTH), class_mode="binary", batch_size=BATCH_SIZE)
    val_gen=val_datagen.flow_from_directory(directory=val_dir, target_size=(IMG_HEIGHT, IMG_WIDTH), class_mode="binary", batch_size=BATCH_SIZE)
    
    #The custom callbacks
    '''The callback will stop iteration when the appointed objectives are reached'''
    class thecallback(tf.keras.callbacks.Callback):
        def __init__(self):
            super(thecallback, self).__init__()
        
        def on_epoch_end(self, epoch, logs=None):
            loggedtrain, loggedval = logs["acc"], logs["val_acc"]
            if loggedtrain > 0.97 and loggedval > 0.97:
                self.model.stop_training = True
                
    #The model!    
    def get_model():
        ## Add dense layers with pre_trained_model.output shape
        x = layers.Flatten()(pre_trained_model.output)
        x = layers.Dense(128, activation="relu")(x)
        x = layers.Dropout(0.2)(x)
        
        ## Binary classiication, use sigmoid!
        x = layers.Dense(1, activation='sigmoid')(x)
        
        ## Merge pre_trained_model with newly made x layers
        model = Model(pre_trained_model.input, x)
        
        model.compile(optimizer=RMSprop(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['acc'])
        return model

    model=get_model()
    model.fit(train_gen, validation_data=val_gen, epochs=10, callbacks=thecallback())

    return model

# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_A3()
    model.save("model_A3.h5")


Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Epoch 1/10


2024-02-15 09:13:02.055666: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-02-15 09:13:02.442031: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-15 09:13:03.125378: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-02-15 09:13:04.456343: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f581cbfe720 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-15 09:13:04.456391: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-02-15 09:13:04.529615: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-15 09:

33/33 [==============================] - 21s 310ms/step - loss: 0.3479 - acc: 0.9299 - val_loss: 0.0075 - val_acc: 1.0000
Epoch 2/10
33/33 [==============================] - 9s 275ms/step - loss: 0.0515 - acc: 0.9815 - val_loss: 0.0204 - val_acc: 0.9922


/root/miniconda3/envs/conda3.9/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
